In [54]:

import pickle

def load_preprocessed_data(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

from process_data import save_image_pkl, save_depth_pkl, load_fish_depth


In [55]:
# saved_folder_path = 'C:\\\\Users\\\\szxkd\\\\Desktop\\\\mmml\\\\camelmera\\\\models\\\\gym\\\\data'
# save_image_pkl(saved_folder_path)

In [56]:
# save_depth_pkl(saved_folder_path)

In [57]:
# image_tensor = load_preprocessed_data("C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\\T1\\image_lcam_fish.pkl")
# depth_tensor = load_preprocessed_data("C:\\Users\\szxkd\\Desktop\\mmml\\camelmera\\models\\gym\\data\\T1\\depth_rcam_fish.pkl")

In [58]:
# print(image_tensor[0].shape)
# print(depth_tensor[0].shape)

In [59]:
# Load Data from Quanting's Folder
saved_folder_path = '/media/jeffrey/2TB HHD/camelmera'
preprocessed_data_file = os.path.join(saved_folder_path, 'prepocessed_imgae_depth_pos_imu_v006.pkl')
data = load_preprocessed_data(preprocessed_data_file)
print(data[0].type)


dict_keys(['observations', 'actions', 'rewards'])
